# IZV Úloha 2 - Testovanie hypotézy
## Hypotéza 1
**Na cestách prvej triedy sa pri nehodách umieralo s rovnakou pravdepodobnosťou ako na cestách tretej triedy.**

In [44]:
# Import the pandas, numby and scipy libraries
import pandas as pd
import scipy.stats as stats
import numpy as np

Načítam dáta do df pomocou read_pickle()

In [45]:
df = pd.read_pickle('accidents.pkl.gz')

Skopírujem dataframe a vytvorím nový stĺpec, ktorý okazuje či boli nehody s obeťami

In [46]:
df1 = df.copy()
df1["fatal"] = df["p13a"] > 0

Vyfiltrujem riadky ktoré ukazujú na nehody na cestách 1. a 3. triedy
Prepíšem hodnoty pomocou map()

In [47]:

df1 = df1[df1["p36"].isin([1, 3])]
df1["p36"] = df1["p36"].map({1: "1st class", 3: "3rd class"})


Vytvorím kontingenčnú tabuľku pomocou crosstab()

In [48]:
tab = pd.crosstab(df1["fatal"], df1["p36"], rownames=["Fatal"], colnames=["Road type"])
print(tab.to_numpy())
tab

[[78618 73352]
 [  911   448]]


Road type,1st class,3rd class
Fatal,,
False,78618,73352
True,911,448


Vypočítam $\chi^2$

In [49]:
chi = stats.chi2_contingency(tab)
chi

(125.72070150000258,
 3.5395243450138555e-29,
 1,
 array([[78824.11109444, 73145.88890556],
        [  704.88890556,   654.11109444]]))

Z rozdielu medzi očakávaným a skutočným výsledkom vidím, že cesty prvej triedy si celkovo vyžiadali viac smrteľných zranení.

In [50]:
tab - chi[3]

Road type,1st class,3rd class
Fatal,,
False,-206.111094,206.111094
True,206.111094,-206.111094


In [51]:
print("P-value", chi[1], "is", "lesser" if chi[1] < 0.05 else "greater", "than 0.05")

P-value 3.5395243450138555e-29 is lesser than 0.05


P-hodnota 3,54*10^-29 je oveľa menšia ako 0,05. Z tohto dôvodu považujem nulovú hypotézu za nepravdivú a viem povedať, že existuje významná korelácia medzi pravdepodobnosťou smrteľných nehôd na cestách prvej a cestách tretej triedy.

## Hypotéza 2
**Pri nehodách vozidiel značky Škoda je škoda na vozidle nižšia ako pri nehodách vozidiel Audi.**

Skopírujem dataframe a vyfiltrujem len riadky, ktoré obsahujú záznamy o vozidlách značky Škoda a Audi

In [52]:
df2 = df.copy()
df_2 = df2[df2["p45a"].isin([39, 2])]

Rozdelím dataframe na dva, každý pre inú značku auta, zresetujem indexy

In [53]:
skoda = df2.loc[df2["p45a"] == 39, ["p53"]].squeeze().reset_index(drop=True)
audi = df2.loc[df2["p45a"] == 2, ["p53"]].squeeze().reset_index(drop=True)


print(skoda.head(), skoda.count())
print("\n")
print(audi.head(), audi.count())


0     200
1     300
2      50
3    1200
4     500
Name: p53, dtype: int64 118379


0    4000
1     400
2     500
3       0
4     200
Name: p53, dtype: int64 11445


In [ ]:
skoda = df2.loc[df2["p45a"] == 39, ["p53"]].squeeze().reset_index(drop=True)
audi = df2.loc[df2["p45a"] == 2, ["p53"]].squeeze().reset_index(drop=True)


print(skoda.head(), skoda.count())
print("\n")
print(audi.head(), audi.count())


0     200
1     300
2      50
3    1200
4     500
Name: p53, dtype: int64 118379


0    4000
1     400
2     500
3       0
4     200
Name: p53, dtype: int64 11445


T-test

In [54]:
val, p = stats.ttest_ind(skoda, audi, equal_var=False, alternative="less")
print("P-value", p, "is", "lesser" if p < 0.05 else "greater", "than set value of 0.05")
print(
    "The value for ",
    "Škoda" if val < 0 else "Audi",
    " is lesser, so in incident damage on ",
    "Škoda" if val < 0 else "Audi",
    " is generally cheaper than on ",
    "Škoda" if val > 0 else "Audi",
    sep="",
)


P-value 6.1078288453876684e-121 is lesser than set value of 0.05
The value for Škoda is lesser, so in incident damage on Škoda is generally cheaper than on Audi


Z výsledku vidím, že P-hodnota je menšia ako 0,05, čo znamená, že nulová hypotéza - priemerné hodnoty dvoch datasetov sú rovnaké, je zamietnutá a že stredná hodnota pre automobily Škoda je menšia. Hypotéza 2 sa teda ukázala ako správna.